In [1]:
sigma_1 = 194 #m2/mol
sigma_2 = 414 #m²/mol
k21 = 0.014 #0.014 s-1
%matplotlib qt
# Objective
# Use non-linear curve fitting to estimate the relaxation rate of an exponential
# decaying signal.

# Steps
# 1. Simulate data (instead of collecting data)
# 2. Define the objective function for the least squares algorithm
# 3. Perform curve fitting
# 4. Compare results

# modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from   scipy import optimize
# 1. Simulate some data
# In the real worls we would collect some data, but we can use simulated data
# to make sure that our fitting routine works and recovers the parameters used
# to simulate the data
def exp_decay(parameters,xdata):
    '''
    Calculate an exponetial decay of the form:
    S= a * exp(-xdata/b)
    '''
    A = parameters[0]
    tau = parameters[1]
    y0 = parameters[2]
    return A * np.exp(-xdata/tau) + y0

def residuals(parameters,x_data,y_observed,func):
    '''
    Compute residuals of y_predicted - y_observed
    where:
    y_predicted = func(parameters,x_data)
    '''
    return func(parameters,x_data) - y_observed


In [2]:
curve = pd.read_csv("../measure_times.csv", names = ['intensity', 'photodiode', 'fluo', 'trigger', 'x_time'], sep = ',', decimal = '.') 
fluo = curve.fluo.values
time_array = curve.x_time.values
        

# Plot the simulated data
plt.plot(time_array,fluo)


FileNotFoundError: [Errno 2] No such file or directory: '../measure_times.csv'

In [14]:
trig = curve.trigger.values
trig_2 = np.roll(trig, 1)
diff = trig-trig_2

trig_down = np.abs(diff) > 1e-2

ticks = np.nonzero(trig_down)[0].tolist()
first = ticks[0]
for second in ticks[1:]:
    print(first, second)
    if second - first < 10:
        ticks.remove(second)
    else: 
        first = second
ticks.append(-1)
print(ticks)
first = ticks[0]
for second in ticks[1:]:
    fluo_transition = fluo[first:second]
    time_transition = time_array[first:second] - time_array[first]
    x0 = [0.1, 0.1, 0.1]
    OptimizeResult  = optimize.least_squares(residuals,  x0, bounds = (-1e5,1e5),
                                          args = (time_transition, fluo_transition, exp_decay))
    parameters_estimated = OptimizeResult.x
    
    # Estimate data based on the solution found
    y_data_predicted = exp_decay(parameters_estimated,time_transition)

    # Plot all together
    plt.figure()
    plt.plot(time_transition, fluo_transition,'ob',
             time_transition,y_data_predicted ,'xr')
    plt.legend(('Real Data','Predicted Data'))

    # How good are the parameters I estimated?
    print( 'Predicted: A, tau, y0 ' + str( parameters_estimated))
    tau = parameters_estimated[1]
    I1 = (1/tau - k21)/sigma_1
    print(I1) #eins/m²/s-1
    first = second

AttributeError: 'DataFrame' object has no attribute 'trigger'

In [48]:
points_per_period = 1000
freq = 2000

end = points_per_period
fluo = curve.fluo.values[:end]
xdata = xdata0[:end]
        

# Plot the simulated data
plt.plot(xdata,fluo)

# 2. Define the objective function for the least squares algorithm
# The scipy.optimize.least_square requires the following inputs

# A) Objective function that computes the residuals of the
#     predicted data vs the observed data using the following syntaxis:
#     f = fun(parameters, *args, **kwargs),



# 3. Perform curve fitting
#    Initial guess for the parameters to be estimated
#    The parameters follow the same order than exp_decay
x0 = [0, 0.1, 0.1]

#    Lower and uppers bounds
lb = [0,0,0]
ub = [10,10,10]

# estimate parameters in exp_decay
OptimizeResult  = optimize.least_squares(residuals,  x0, bounds = (0,10),
                                          args = (xdata, fluo, exp_decay))
parameters_estimated = OptimizeResult.x

# Estimate data based on the solution found
y_data_predicted = exp_decay(parameters_estimated,xdata)

# Plot all together
plt.figure(2)
plt.plot(xdata,fluo,'ob',
         xdata,y_data_predicted ,'xr')
plt.legend(('Real Data','Predicted Data'))

# How good are the parameters I estimated?
print( 'Predicted: A, tau, y0 ' + str( parameters_estimated))
tau = parameters_estimated[1]
I1 = (1/tau - k21)/sigma_1
print(I1) #eins/m²/s-1

Predicted: A, tau, y0 [0.44422742 0.24879966 0.02846871]
0.0206458667010828


In [28]:
def exp_up(parameters,xdata):
    '''
    Calculate an exponetial decay of the form:
    S= a * exp(-xdata/b)
    '''
    A = parameters[0]
    tau = parameters[1]
    y0 = parameters[2]
    return -A * np.exp(-xdata/tau) + y0

x0 = [-0.52 * 0, 0.11, 0.69]

high = points_per_period
high_end = 2 * points_per_period
fluo = curve.fluo.values[high:high_end]
xdata = xdata0[high:high_end] - points_per_period/freq

# estimate parameters in exp_decay
OptimizeResult  = optimize.least_squares(residuals,  x0, bounds = (-1e6,1e6),
                                          args = (xdata, fluo, exp_decay))
parameters_estimated = OptimizeResult.x

# Estimate data based on the solution found
y_data_predicted = exp_decay(parameters_estimated,xdata)

# Plot all together
plt.figure(2)
plt.plot(xdata,fluo,'ob',
         xdata,y_data_predicted ,'xr')
plt.legend(('Real Data','Predicted Data'))

print( 'Predicted: A, tau, y0 ' + str( parameters_estimated))
tau2 = parameters_estimated[1]
I2 = (1/tau2 - sigma_1*I1 - k21)/sigma_2
print(I2) #eins/m²/s-1

Predicted: A, tau, y0 [-1.56554966e+02  8.58179637e-02  5.22974782e-01]
0.018437859523734754


In [5]:
points_per_period = 10000
freq = 200000
num_period = 2
acq_time = 4

rising = []
falling = []
xdata = np.linspace(0, acq_time/num_period/2, points_per_period)
for k in range(2):
    rising.append(curve.fluo.values[])


SyntaxError: invalid syntax (<ipython-input-5-f7bef9ad29b7>, line 10)

In [25]:
ticks = np.nonzero(trig_down)[0].tolist()
first = ticks[0]
for second in ticks[1:]:
    print(first, second)
    if second - first < 10:
        ticks.remove(second)
    else: 
        first = second
        
print(ticks)

0 1
0 1000
1000 1001
1000 2000
2000 2001
2000 3000
3000 3001
[0, 1000, 2000, 3000]


In [82]:
curve = pd.read_csv("..\\check_bode\\2020-06-30_16_30_Bode_Diagram_intensity_large_span\\2020-06-30_17_29_exponential_fit_85004.csv", sep = ',', decimal = '.') 
K = curve.keys()
time_array = curve[K[1]].values
fluo = curve[K[2]].values
     
x0 = [-0.52 * 0, 0.11, 0.69]


# estimate parameters in exp_decay
OptimizeResult  = optimize.least_squares(residuals,  x0, bounds = (-1e6,1e6),
                                          args = (time_array, fluo, exp_decay))
parameters_estimated = OptimizeResult.x

# Estimate data based on the solution found
y_data_predicted = exp_decay(parameters_estimated,time_array)

# Plot all together
plt.figure(2)
plt.plot(time_array,fluo,'ob',
         time_array,y_data_predicted ,'xr')
plt.legend(('Real Data','Predicted Data'))
print(1/parameters_estimated[1])

24.095588660829108
